In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#with open(r"C:\Users\Francesco\Desktop\The Psychology of Totalitarianism 2022- Mattias Desmet.txt", encoding='utf-8') as f:
#    book = f.read()

#loader = [UnstructuredFileLoader(r"C:\Users\Francesco\Desktop\The Psychology of Totalitarianism 2022- Mattias Desmet.pdf", encoding='utf-8'), UnstructuredFileLoader(r"C:\Users\Francesco\Desktop\The unhappiness machine.pdf", encoding='utf-8')]
#raw_documents = AnalyticDB.from_documents(loader, )

loader = DirectoryLoader(r"C:\Users\Francesco\Desktop\books", silent_errors=True, show_progress=True)
raw_documents = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap  = 20)
documents = text_splitter.split_documents(raw_documents)

docsearch = Chroma.from_documents(
    documents, embeddings, persist_directory="db", metadatas=[{"source": f"{i}-pl"} for i in range(len(documents))]
)
docsearch.persist()
docsearch = None

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import DirectoryLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docsearch = Chroma(persist_directory="db", embedding_function=embeddings)

Using embedded DuckDB with persistence: data will be stored in: db


In [9]:
## query the database using similarity search


query = "How can i organize the best dinner for getting people together?"

if query == "exit":
    pass
else:
    docs = docsearch.similarity_search_with_score(query, k=3)
    for doc in docs:
        print(doc[0].page_content)
        print(doc[0].metadata["source"])

5. Forget being formal.

Most dinner parties don't call for anything fancy. Follow the KISS principle (Keep It Simple, Silly). Good food. Good people. Lots of wine. Good conversation. That's a successful dinner party. I always underdress just so no one else feels they did. Jeans and a jacket are my standard fare, but you judge for yourself.

6. Don't seat couples together.

The essence of a good dinner party lies in seating everyone prop erly. If you seat couples together, things can get boring. Mix and

200

Never Eat Alone

match,  putting  people  together  who  don't  know  each  other  but perhaps share  an  interest of some kind.  I like to  set placeholders where I want people to sit. Each placeholder is a simple card with the guest's name on it. If I have the time, I love to put an interest ing question or joke on the back of the card that guests can use to break the ice with one another. Or you can go out and buy funny greeting cards just to make things interesting.

7. Relax.

In [5]:
from langchain.chains import RetrievalQA
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma

llm = HuggingFacePipeline.from_model_id(model_id="bigscience/bloom-1b1", task="text-generation", model_kwargs={"temperature":0, "max_length":1000})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(), return_source_documents=True)

In [12]:
qa({"query": "summarize: How can i organize the best dinner for getting people together? Sommarize in 5 points"}, return_only_outputs=True)


{'result': '\n\n1. Set a theme. Set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party, you can set a theme for the dinner. For example, if you are going to a dinner party',
 'source_documents': [Document(page_content="5. Forget being formal.\n\nMost dinner parties don't call for anything fancy. Follow the KISS principle (Keep It Simple, Silly). Good food. Good people. Lots of wine. Good conversation. That's a successful dinner party. I always underdress just so no